In [385]:
# ! pip install get_nba_data
# ! pip install nba_py
# ! pip install nba_api
import requests
import json
import pandas
import numpy
#import get_nba_data
#import nba_py
import nba_api

In [386]:
from nba_api.stats.endpoints import leaguedashplayerstats
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2018-19')
catstats2018_2019 = playerinfo.get_data_frames()
catstats2018_2019 = pandas.concat(catstats2018_2019)
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2017-18')
catstats2017_2018 = playerinfo.get_data_frames()
catstats2017_2018 = pandas.concat(catstats2017_2018)
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2016-17')
catstats2016_2017 = playerinfo.get_data_frames()
catstats2016_2017 = pandas.concat(catstats2016_2017)

In [387]:
final2018_2019 = catstats2018_2019[['PLAYER_ID','PLAYER_NAME','TEAM_ABBREVIATION','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
 'OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2018_2019 = final2018_2019.add_suffix('_2018')
final2017_2018 = catstats2017_2018[['PLAYER_ID','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
 'OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2017_2018 = final2017_2018.add_suffix('_2017')
final2016_2017 = catstats2016_2017[['PLAYER_ID','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
 'OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2016_2017 = final2016_2017.add_suffix('_2016')
final = final2018_2019.merge(final2017_2018, left_on='PLAYER_ID_2018', right_on='PLAYER_ID_2017', how='left')
final = final.merge(final2016_2017, left_on='PLAYER_ID_2018', right_on='PLAYER_ID_2016', how='left')
final['FTMiss_2018'] = final.apply(lambda row: row.FTA_2018 - row.FTM_2018, axis=1)
final['FTMiss_2017'] = final.apply(lambda row: row.FTA_2017 - row.FTM_2017, axis=1)
final['FTMiss_2016'] = final.apply(lambda row: row.FTA_2016 - row.FTM_2016, axis=1)
final['FG3Miss_2018'] = final.apply(lambda row: row.FG3A_2018 - row.FG3M_2018, axis=1)
final['FG3Miss_2017'] = final.apply(lambda row: row.FG3A_2017 - row.FG3M_2017, axis=1)
final['FG3Miss_2016'] = final.apply(lambda row: row.FG3A_2016 - row.FG3M_2016, axis=1)
final['FG2Made_2018'] = final.apply(lambda row: row.FGM_2018 - row.FG3M_2018, axis=1)
final['FG2A_2018'] = final.apply(lambda row: row.FGA_2018 - row.FG3A_2018, axis=1)
final['FG2Miss_2018'] = final.apply(lambda row: row.FG2A_2018 - row.FG2Made_2018, axis=1)
final['FG2Made_2017'] = final.apply(lambda row: row.FGM_2017 - row.FG3M_2017, axis=1)
final['FG2A_2017'] = final.apply(lambda row: row.FGA_2017 - row.FG3A_2017, axis=1)
final['FG2Miss_2017'] = final.apply(lambda row: row.FG2A_2017 - row.FG2Made_2017, axis=1)
final['FG2Made_2016'] = final.apply(lambda row: row.FGM_2016 - row.FG3M_2016, axis=1)
final['FG2A_2016'] = final.apply(lambda row: row.FGA_2016 - row.FG3A_2016, axis=1)
final['FG2Miss_2016'] = final.apply(lambda row: row.FG2A_2016 - row.FG2Made_2016, axis=1)
final.drop(['FG3A_2018', 'FG2A_2018', 'FTA_2018', 'FGM_2018', 'FGA_2018',
            'FG3A_2017', 'FG2A_2017', 'FTA_2017', 'FGM_2017', 'FGA_2017', 
            'FG3A_2016', 'FG2A_2016', 'FTA_2016', 'FGM_2016', 'FGA_2016', 
            'PLAYER_ID_2017', 'PLAYER_ID_2016'], axis=1, inplace=True)
final = final.rename(index=str,columns={'PLAYER_ID_2018': 'PLAYER_ID',
                                        'PLAYER_NAME_2018':'PLAYER',
                                        'TEAM_ABBREVIATION_2018':'TEAM',
                                        'FTM_2018':'FTMade_2018', 'FTM_2017':'FTMade_2017', 'FTM_2016':'FTMade_2016',
                                        'FG3M_2018':'FG3Made_2018', 'FG3M_2017':'FG3Made_2017', 'FG3M_2016':'FG3Made_2016'})

In [388]:
from nba_api.stats.endpoints import leaguedashplayerbiostats
playerinfo = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
PlayerBios = playerinfo.get_data_frames()
PlayerBios = pandas.concat(PlayerBios)
PlayerBios = PlayerBios[['PLAYER_ID','AGE','PLAYER_HEIGHT_INCHES','PLAYER_WEIGHT','COUNTRY','DRAFT_NUMBER']]
PlayerBios['DRAFT_NUMBER'] = PlayerBios.DRAFT_NUMBER.replace('Undrafted','0')
PlayerBios['DRAFT_NUMBER'] = PlayerBios['DRAFT_NUMBER'].astype(str).astype(int)
PlayerBios = PlayerBios.rename(index=str,columns={'DRAFT_NUMBER': 'DRAFT'})
final = final.merge(PlayerBios, left_on='PLAYER_ID', right_on='PLAYER_ID', how='left')

In [389]:
Sals2018 = pandas.read_csv('2018Sals.csv', dtype={'CAP HIT':float})
Sals2018 = Sals2018.rename(index=str,columns={'CAP HIT': 'PREV_SAL'})
final = final.merge(Sals2018, left_on='PLAYER', right_on='PLAYER', how='inner')

In [390]:
Sals2019 = pandas.read_csv('2019Sals.csv', dtype={'CAP HIT':float})
Sals2019 = Sals2019.rename(index=str,columns={'CAP HIT': 'SAL'})
Signed = final.merge(Sals2019, left_on='PLAYER', right_on='PLAYER', how='inner')

In [391]:
FAs2019 = pandas.read_csv('FAs2019.csv')
FAs2019 = FAs2019.rename(index=str,columns={'TEAM': 'PREV_TEAM'})
FAs = final.merge(FAs2019, left_on='PLAYER', right_on='PLAYER', how='inner')

In [392]:
Signed['PLAYER_WEIGHT'] = Signed.PLAYER_WEIGHT.astype(numpy.int64)
FAs['PLAYER_WEIGHT'] = FAs.PLAYER_WEIGHT.astype(numpy.int64)

In [393]:
Signed.loc[Signed['COUNTRY'] != 'USA', 'COUNTRY'] = '0'
Signed.loc[Signed['COUNTRY'] == 'USA', 'COUNTRY'] = '1'
FAs.loc[FAs['COUNTRY'] != 'USA', 'COUNTRY'] = '0'
FAs.loc[FAs['COUNTRY'] == 'USA', 'COUNTRY'] = '1'

In [394]:
Signed.loc[Signed['POS'] == 'PG', 'POS'] = '1'
Signed.loc[Signed['POS'] == 'SG', 'POS'] = '2'
Signed.loc[Signed['POS'] == 'SF', 'POS'] = '3'
Signed.loc[Signed['POS'] == 'PF', 'POS'] = '4'
Signed.loc[Signed['POS'] == 'C', 'POS'] = '5'
FAs.loc[FAs['POS'] == 'PG', 'POS'] = '1'
FAs.loc[FAs['POS'] == 'SG', 'POS'] = '2'
FAs.loc[FAs['POS'] == 'SF', 'POS'] = '3'
FAs.loc[FAs['POS'] == 'PF', 'POS'] = '4'
FAs.loc[FAs['POS'] == 'C', 'POS'] = '5'

In [395]:
Signed['COUNTRY'] = Signed.COUNTRY.astype(numpy.int64)
Signed['POS'] = Signed.POS.astype(numpy.int64)
Signed['DRAFT'] = Signed.DRAFT.astype(numpy.int64)
FAs['COUNTRY'] = FAs.COUNTRY.astype(numpy.int64)
FAs['POS'] = FAs.POS.astype(numpy.int64)
FAs['DRAFT'] = FAs.DRAFT.astype(numpy.int64)

In [399]:
Signed = Signed.fillna(0)
FAs = FAs.fillna(0)

In [400]:
import ipywidgets as widgets
from ipywidgets import interact
def view(Team):
    return Signed[Signed['TEAM'] == Team]

team = widgets.Dropdown(
    options = list(sorted(Signed['TEAM'].unique())),
    description = 'Team: '
)

interact(view, Team = team)

interactive(children=(Dropdown(description='Team: ', options=('ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL',…

<function __main__.view(Team)>